# 16. 복잡한 계산 그래프(구현 편)
- 이번 단계에서는 15단계에서 설명한 이론을 코드로 구현합니다. 
- 가장 먼저 순전파 시 '세대'를 설정하는 부분부터 시작하겠습니다. 
- 그런 다음 역전파 시 최근 세대의 함수부터 꺼내도록 합니다. 
- 이렇게 하면 아무리 복잡한 계산 그래프라도 올바른 순서로 역전파가 이루어집니다. 

## 16.1 세대 추가
- 먼저 Variable 클래스와 Function 클래스에 인스턴스 변수 generation을 추가하겠습니다. 
- 몇 번째 '세대'의 함수(혹은 변수)인지 나태내는 변수입니다. 
- Variable 클래스부터 시작하겠습니다. 

In [36]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0       # 세대 수를 기록하는 변수, 0으로 초기화

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1      # 세대를 기록한다(부모 함수의 세대보다 1만큼 큰 값을 설정)

- f.generation이 2인 함수에서 만들어진 변수인 y의 generation은 3이 됩니다. 

![title](16-1.png)

- 다음 차례는 Function 클래스입니다. 
- Function 클래스의 generation은 입력 변수와 같은 값으로 설정합니다. 
- 외쪽처럼 입력 변수의 generation이 4라면 함수의 generation도 4가 됩니다. 

![title](16-2.png)

- 입력 변수가 둘 이상이라면 가장 큰 generation의 수를 선택합니다. 

In [30]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])     # 입력 변수가 여러개라면 가장 큰 generation이 수를 선택
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

## 16.2 세대 순으로 꺼내기
- 지금까지의 수정을 반영하여 일반적인 계산(순전파)을 하면 모든 변수와 함수에 세대가 설정됩니다. 

![title](16-3.png)

- 이렇게 세대가 설정되어 있으면 역전파 때 함수를 올바른 순서로 꺼낼 수 있습니다. 
- Variable 클래스의 backward 메서드 안에서는 처리할 함수의 후보들을 funcs 리스트에 보관합니다. 
- 따라서 funcs에서 세대가 큰 함수부터 꺼내게 하면 올바른 순서로 역전파할 수 있습니다. 

In [31]:
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
    f = Function()   # 더미 함수 클래스
    f.generation = g
    funcs.append(f)
    
[f.generation for f in funcs]

[2, 0, 1, 4, 2]

In [32]:
funcs.sort(key=lambda x: x.generation)   # 리스트의 원소를 x라고 했을 때, x.generation을 키로 사용해 정렬
[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [33]:
f = funcs.pop()     # 정렬 후 리스트의 가장 끝 원소를 꺼내면 가장 큰 값을 꺼낸다. 우선순위 큐, heapq로 구현 가능
f.generation

4

heapq 참고자료] : https://yhmin84.medium.com/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%9A%B0%EC%84%A0%EC%88%9C%EC%9C%84-%ED%81%90-priority-queue-%EB%A5%BC-%EC%9C%84%ED%95%9C-heapq-%EB%AA%A8%EB%93%88-%EC%82%AC%EC%9A%A9%EB%B2%95-b33c4e0ef2b1

# 16.3 Variable 클래스의 backward
- Variable 클래스의 backward 구현해 보겠습니다.  

In [34]:
class Variable:
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()            # funcs 리스트에 같은 함수를 중복 추가하는 일을 막기 위해서 집합(set)을 이용하고 있다

        def add_func(f):            # 함수 리스트를 세대 순으로 정렬하는 역할, 중첩함수로 정의
            if f not in seen_set:   # 중첩 함수는 감싸는 메서드(backward 메서드) 안에서만 이용한다
                funcs.append(f)     # 중첩 함수는 감싸는 메서드(backward 메서드)에 정의된 변수(funcs, seen_set)를 사용해야 한다
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()                   # 자동으로 세대가 가장 큰 함수를 꺼내게 됨
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)       # 수정 전: funcs.append(x.creator)

# 16.4 동작 확인
- 아래 계산을 미분해봅시다.

![title](16-4.png)

- 코드로는 다음과 같습니다. 


In [41]:
import numpy as np

x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


- 이제 아무리 복잡한 '연결'도 제대로 미분할 수 있습니다.

![title](16-5.png)

In [38]:
# 16장을 전체 코드로 정리한 버전

import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0       # 세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1      # 세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


# 17. 메모리 관리와 순환 참조

- 이번 단계와 다음 단계에서 성능 개선 대책(기술)을 도입할 계획입니다. 
- 파이썬에서의 메모리 관리에 대해서 알아봅시다. 

## 17.1 메모리 관리

- 파이썬은 필요 없어진 객체를 메모리에서 자동으로 삭제합니다. 
- 이 고마운 기능 덕에 우리는 메모리 관리를 의식할 일이 크게 줄어듭니다. 
- 불필요한 객체는 파이썬 인터프리터가 제거해주기 때문에 우리는 더 중요한 작업에 집중할 수 있는 것이죠.
- 그렇더라도 코드를 제대로 작성하지 않으면 때때로 메모리 누수(momoey leak) 또는 메모리 부족(out of memory) 등의 문제가 발생합니다. 
- 특히 신경망에서는 큰 데이터를 다루는 경우가 많아서 메모리 관리를 제대로 하지 않으면 실행시간이 오래 걸리는 (GPU의 경우 실행할 수조차 없는) 일이 자주 발생합니다.
- 파이썬에는 아래 2가지 방법으로 메모리를 관리합니다. 
    - 참조(reference) 카운트: 참조 수를 세는 방식
    - 가비지 콜렉션(Garbage Collections): 세대(generation)를 기준으로 '쓸모없어진 객체'(garbage)를 회수하는 방식
    
## 17.2 참조 카운트 방식의 메모리 관리

- 모든 객체는 참조 카운트가 0인 상태로 생성되고, 아래 경우에 참조 카운트가 증가합니다. 
    - 대입 연산자를 사용할 때
    - 참수에 인수로 전달할 때
    - 컨테이너 타입 객체(리스트, 튜플, 클래스 등)에 추가할 때
    
- 반대로 객체에 대한 참조가 끊길 때마다 1만큼 감소하다가 0이 되면 파이썬 인터프리터가 회수해갑니다. 
    - 이런 방식으로 객체가 더 이상 필요 없어지면 즉시 메모리에서 삭제됩니다. 

In [43]:
class obj:
    pass

def f(x):
    print(x)
    
a = obj      # 변수에 대입: 참조 카운트 1
f(a)         # 함수에 전달: 함수 안에서는 참조 카운트 2
             # 함수 완료: 빠져나오면 참조 카운트 1
a = None     # 대입 해제: 참조 카운트 0

<class '__main__.obj'>


In [44]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c            # 왼쪽 그림

a = b = c = None   # 오른쪽 그림

![title](17-1.png)
- a, b, c라는 세 개의 객체를 생성했습니다. 
- a가 b를 참조하고, b가 c를 참조합니다. 
- 객체의 관리는 왼쪽처럼 되었습니다. 
- a = b = c = None 실행하면 오른쪽 처럼 변합니다. 
- 이때 a의 참조 카운트는 0이 되므로, 즉시 삭제됩니다. 
- b와 c의 참조 카운트는 1인데, a가 삭제되면서 0으로 감소하여 역시 삭제됩니다. 
- 이렇게 사용자로부터 참조되지 않는 객체들이 마치 도미노처럼 한꺼번에 삭제되는 것입니다. 
- 하지만, 참조 카운트로는 해결할 수 없는 문제가 있습니다. 바로 순환 참조입니다. 

## 17.3 순환 참조

In [46]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c  
c.a = a            # 왼쪽 그림

a = b = c = None   # 오른쪽 그림

- 이번에 c에서 a로의 참조가 추가되었습니다. 
- 세 개의 객체가 원 모양을 이루며 서로가 서로를 참조하게 되는데, 이 상태가 바로 순환 참조입니다. 

![title](17-2.org)

- 오른쪽 그램에서 a, b, c의 참조 카운트는 모두 1입니다. 
- 하지만 사용자는 이들 세 객체 중 어느 것에도 접근할 수 없습니다. (즉, 모두 불필요한 객체입니다.)
- a = b = c = None을 실행하는 것으로는 순환 참조의 참조 카운트가 0이 되지 않고, 결과적으로 메모리에서 삭제되지 않습니다. 
- 그래서 또 다른 메모리 관리 방식으로 'GC'(세대별 가비지 컬렉션, 'generational garbage collection')입니다. 
- GC는 참조 카운트보다 영리한 방식으로 불필요한 객체를 찾아냅니다. 메모리가 부족해지는 시점에 파이썬 인터프리터에 의해 자동으로 호출됩니다. 
- 물론 명시적으로 호출할 수도 있습니다. (gc 모둘을 임포트해서 gc.collect()를 실행)
- GC는 순환 참조를 올바르게 처리합니다. 하지만 메모리 해제를 GC에 미루다 보면 프로그램 전체 메모리 사용량이 커지는 원인이 됩니다. 
- 머신러닝, 신경망에서 메모리는 중요한 자원입니다. 따라서 순환 참조를 만들지 않는 것이 좋습니다!
- 현재의 DeZero에는 순환 참조가 존재합니다. 바로 아래와 같이 '변수'와 '함수'를 연결하는 방식에 순환 참조가 숨어 있습니다. 

![title](17-3.png)

- Function 인스턴스는 두 개의 Variable 인스턴스(입력과 출력)를 참조합니다. 
- 그리고 출력 Variable 인스턴스는 창조자인 Fuction 인스턴스를 참조합니다. 
- 이 때 Function 인스턴스와 Variable 인스턴스가 순환 참조 관계를 만듭니다. 
- 다행히 이 순환 참조는 표준 파이썬 모듈인 weakrdf로 해결할 수 있습니다. 

# 17.4 weakref 모듈

- 파이썬에서는 weakref 함수를 사용하여 약한 참조(weak reference)를 만들 수 있습니다. 
- 약한참조란 다른 객체를 참조하되 참조 카운트는 증가시키지 않는 기능입니다. 

In [47]:
import weakref
import numpy as np

a = np.array([1, 2, 3])   # 일반적인 방식의 참조
b = weakref.ref(a)        # 약한 참조

b

<weakref at 0x7f0834771170; to 'numpy.ndarray' at 0x7f08363ddcb0>

In [48]:
b()

array([1, 2, 3])

- 위는 ndarry 인스턴스를 대상으로 실험을 해봤습니다. 
- b를 출력해보면 ndarry를 가리키는 약한 참조(weakref)임을 확인할 수 있습니다. 
- 참조된 데이터에 접근하여면 b()라고 쓰면 됩니다. 

In [49]:
a = None
b               # <weakref at 0x103b7f048; dead>

<weakref at 0x7f0834771170; to 'numpy.ndarray' at 0x7f08363ddcb0>

- ndarray 인스턴스는 참조 카운트 방식에 따라 메모리에서 삭제됩니다. 
- b도 참조를 가지고 있지만 약한 참조이기 때문에 참조 카운트에 영향을 주지 못하는 것입니다. 
- 그래서 b를 출력하면 dead라는 문자가 나옵니다. 이것으로 ndarray 인스턴스가 삭제됐음을 알 수 있습니다. 
- 파이썬 인터프리터에서 실행하면 dead가 나오지만, 
- IPython과 쥬피터 노트북 등의 인터프리터 자체가 사용자가 모르는 참조를 추가로 유지하기 때문에 
- 앞의 코드의 b가 여전히 유효한 참조를 유지할 것입니다. 
- weakref 구조를 DeZero에도 도입하면 아래와 같습니다. 

In [50]:
import weakref

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]  # self.output이 대상을 약한 참조로 가리키게 변경
        return outputs if len(outputs) > 1 else outputs[0]
    
class Variable:
    """..."""    
    def backward(self):
        """..."""￼

        while funcs:
            f = funcs.pop()
            # 수정전: gys = [output.grad for output in f.outputs]
            gys = [output().grad for output in f.outputs]  # output is weakref 
        """..."""

## 17.5 동작 확인

- 순환 참조가 없어진 새로운 DeZero에서 다음 코드를 실행해보죠

In [52]:
for i in range(10):
    x = Variable(np.random.randn(10000))   # 거대한 데이터
    y = square(square(square(x)))          # 복잡한 계산을 수행한다

- for문을 사용하여 계산을 반복해 수행했습니다. 
- 이 반복문은 아래와 같이 복잡한 참조 구조를 만들어냅니다. 

![title](17-4.png)

- for문이 두 번째 반복될 때 x와 y가 덮어 써집니다. 
- 그러면 사용자는 이전의 계산 그래프를 더 이상 참조하지 않게 되죠.
- 참조 카운트가 0이 되므로 이 시전에 계산 그래프에 사용된 메모리가 바로 삭제됩니다. 
- 이것으로 DeZero 순환 참조 문제가 해소되었습니다. 
- 파이썬으로 메모리 사용량을 측정하려면 외부 라이브러리인 memory profiler 등을 사용하면 편리합니다. 
- 방금 전의 코드를 실제로 측정해보면 메모리 사용량이 전혀 증가하지 않음을 확인할 수 있을 겁니다. 

In [57]:
# 17장을 전체 코드로 정리한 버전


import weakref
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


for i in range(10):import weakref
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


for i in range(10):
    x = Variable(np.random.randn(10000))  # big data
    y = square(square(square(x)))
    x = Variable(np.random.randn(10000))  # big data
    y = square(square(square(x)))

# 18. 메모리 절약 모드

- DeZero의 메모리 사용을 개선할 수 있는 구조 두 가지를 도입합니다. 
    - 첫 번째는 역전파 시 사용하는 메모리양을 줄이는 방법으로, 불필요한 미분 결과를 보관하지 않고 즉시 삭제합니다. 
    - 두 번째는 '역전파가 필요 없는 경우용 모드'를 제공하는 것입니다. 이 모드에서는 불필요한 계산을 생략합니다. 
    
## 18.1 필요 없는 미분값 삭제

- 첫 번째로 DeZero의 역전파를 개선하겠습니다. 
- 현재의 DeZero에서는 모든 변수가 미분값을 변수에 저장해두고 있습니다. 

In [58]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

1.0 1.0
2.0 1.0


- 사용자가 제공한 변수는 x0와 x1이며, 다른 변수 t와 y는 계산 결과로 만들어집니다. 
- y.backward()를 실행하여 미분하면 모든 변수가 미분 결과를 메모리에 유지합니다. 
- 역전파로 구하고 싶은 미분값은 말단 변수(x0, x1)뿐 일 때가 대부분입니다. 
- 앞의 에에서는 y와 t같은 중간 변수의 미분값은 필요하지 않습니다. 
- 그래서 중간 변수에 대해서는 미분값을 제거하는 모드를 추가하겠습니다. 

In [59]:
class Variable:
...
    def backward(self, retain_grad=False):  # 메서드 인수에 retain_grad 추가, True이면 모든 변수가 미분 결과(기울기) 유지
        if self.grad is None:               
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:      # False(기본값)이면 중간 변수의 미분값을 모두 None으로 재설정
                for y in f.outputs:  # 이렇게 하면 말단 변수 외에는 미분값을 유지하지 않음
                    y().grad = None  # y is weakref, y()라고 한 이유는 y가 약한 참조이기 때문, 
                                     # 이 코드가 실행되면 참조 카운트가 0이 되어 미분값 데이터가 메모리에서 삭제됨

In [63]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)       # 중간 변수인 y와 t의 미분값이 삭제되어 그만큼의 메모리를 다른 용도로 사용할 수 있게 됨
print(x0.grad, x1.grad)

None None
2.0 1.0


## 18.2 Function 클래스 복습

- DeZero에서 미분을 하려면 순전파를 수행한 뒤 역전파해주면 됩니다. 
- 그리고 역전파 시에는 순전파의 계산 결과가 필요하기 때문에 순전파 때 결과값을 기억해둡니다. 
- 결과값을 보관하는 로직은 바로 Function 클래스의 아래 부분입니다 .

In [64]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs            # 순전파 때 결과값을 보관하는 부분
        self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

- 함수는 입력을 inputs라는 '인스턴스 변수'로 참조합니다. 
- inputs가 참조하는 변수의 참조 카운트가 1만큼 증가하고, $__call__$메서드에서 벗어난 뒤에도 메모리에 생존합니다. 
- 만약 인스턴스 변수인 inputs로 참조하지 않았다면 참조 카운트가 0이 되어 메모리에서 삭제됐을 것입니다. 
- 인스턴스 변수 inputs는 역전파 계산 시 사용됩니다. 따라서 역전파하는 경우라면 참조할 변수들을 inputs에 미리 보관해둬야 합니다. 
- 하지만 때로는 미분값이 필요 없는 경우도 있습니다. 
- 이런 경우라면 중간 계산 결과를 저장할 필요가 없고, 계산의 '연결' 또한 만들 이유가 없습니다. 
- 신경망에는 **학습(또는 훈련, training)** 과 **추론(inference)** 라는 두 가지 단계가 있습니다. 
- 학습 시에는 미분값을 구해야 하지만, 추론 시에는 단순히 순전파만 하기 때문에 중간 계산 결과를 곧바로 버리면 메모리 사용량을 크게 줄일 수 있습니다. 

## 18.3 Config 클래스를 활용한 모드 전환

- 순전파만 할 경우를 위한 개선을 DeZero에 추가하겠습니다. 
- 우선 두가지 모드, 즉 '역전파 활성 모드'와 '역전파 비활성 모드'를 전환하는 구조가 필요합니다. 
- 간단히 다음 Config 클래스를 이용할 것입니다. 
    - 설정 데이터는 단 한군데에만 존재하는 게 좋습니다. 
    - 그래서 Config 클래스는 인스턴스화 하지 않고 '클래스' 상태로 이용합니다. 
    - 인스턴스는 여러 개 생성할 수 있지만, 클래스는 항상 하나만 존재하기 때문입니다. 
    - 따라서 아래 코드에서 Config 클래스가 '클래스 속성'을 갖도록 설정했습니다. 

In [65]:
class Config:
    enable_backprop = True     # True 이면 역전파 활성 모드이고 False 이면 역전파 비활성 모드임

- Config 클래스를 정의했으니 Function에서 참조하게 하여 모드를 전환할 수 있게 합니다. 

In [66]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:    # Config.enable_backprop이 True일 때만 역전파 코드가 실행됨
            self.generation = max([x.generation for x in inputs]) # '세대'는 역전파 시 노드를 따라가는 순서를 정하는데 사용
            for output in outputs:
                output.set_creator(self)  # 계산들의 연결을 만드는 부분으로 역전파 비활성 모드에서는 필요없음 
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

## 18.4 모드 전환

- 역전파 활성/비활성을 구분 짓는 구조가 만들어졌습니다. 
- 이 구조를 활용하면 다음과 같이 모드를 전환할 수 있습니다. 

In [ ]:
# True이면 중간 계산 결과가 (적어도 역전파가 완료되기 전까지는) 계속 유지되어 그만큼 메모리를 차지합니다. 
config.enable_backprop = True          
x = Variable(np.ones((100, 100, 100))
y = square(square(square(x)))            
y.backward()
             
# False이면 중간 계산 결과는 사용 후 곧바로 삭제됩니다. 정확하게는 다른 객체에서 참조가 없어지는 시점에 메모리에서 삭제됩니다. 
config.enable_backprop = False         
x = Variable(np.ones((100, 100, 100))
y = square(square(square(x)))            

## 18.5 with 문을 활용한 모드 전환

- 파이썬에서는 with라고 하는, 후처리를 자동으로 수행하고자 할 때 사용할 수 있는 구문이 있습니다. 
- 대표적인 예는 파일의 open과 close입니다. 

In [ ]:
# with 문 없이 파일에 무언가를 쓰려면 다음처럼 작성해야 합니다. 
f = open('sample.txt', 'w')
f.write('hello world!')
f.close

In [ ]:
# open()으로 파일을 열고, 무언가를 쓰고, close()로 파일을 닫습니다. 매번 close()하기도 귀찮고 실수할 수 있으므로 with 문을 사용합니다. 
with open('sample.txt', 'w') as f:
    f.write('hello world!')

- with 블록에 들어갈 때 파일이 열리고, with 블록 안에서 파일은 계속 열린 상태고 블록을 빠져나올 때 (사용자에게 보이지 않는 곳에서) 자동으로 닫힙니다. 
- 이와 같이 with 문을 사용하는 것으로 'with 블록을 들어갈 때의 처리(전처리)'와 'with 블록을 빠져나올 때의 처리(후처리)'를 자동으로 할 수 있습니다. 
- 이러한 with 문의 원리를 이용하여 '역전파 비활성 모드'로 전환하려 합니다. 
    - '역전파 비활성 모드'로 일시적으로 전환하는 방법은 실전에서 자주 사용됩니다. 
    - 신경망 학습에서는 모델 평가를(학습 도중에) 하기 위해 '역전파'가 필요 없는 모드를 사용하는 일이 자주 발생합니다. 

In [ ]:
with using_config('enable_backprop', False):   # with 블록 안에서만 '역전파 비활성 모드'가 됩니다. 
    x = Variable(np.array(2.0))
    y = square(x)
    
# with 블록을 벗어나면 일반 모드, 즉 '역전파 활성 모드'로 돌아갑니다. 

In [68]:
import contextlib

@contextlib.contextmanager     # 데코레이터를 달면 문맥(context)을 판단하는 함수가 만들어 짐
def config_test():
    print('start')   # 전처리
    try:                       # with 블록 안에서 예외가 발생할 수 있고, 발생한 예외는 yield를 실행하는 코드로 전달됩니다. 
        yield                  # yield 전에는 전처리 로직을, yield 다음에는 후처리 로직을 작성함
    finally:                   # with 블록에서 발생한 예외 처리를 위해서 yield는 try/finally로 감싸야 합니다. 
        print('done')  # 후처리
        
with config_test():            # with 블록 안으로 들어갈 때 전처리가 실행되고, 블록 범위를 빠져나올 때 후처리가 실행됩니다. 
    print('process...')

start
process...
done


In [69]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):  # name은 타입이 str이며, 사용할 Config 속성의 이름(클래스 속성 이름)을 가리킵니다. 
    old_value = getattr(Config, name)  # name을 getattr 함수에 넘겨 Config 클래스에서 꺼내옵니다. 
    setattr(Config, name, value) # 그런 다음 setattr 함수를 사용하여 새로운 값을 설정합니다. 
    try:
        yield
    finally:
        setattr(Config, name, old_value)  

In [ ]:
# with 블록을 들어갈 때 name으로 지정한 Config 클래스 속성이 value로 설정됩니다. with 블록을 빠져나오면서 원래 값(old_value)로 복원
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

- 역전파가 필요없는 경우에는 with 블록에서 순전파 코드만 실행합니다. 
- 이제 불필요한 계산을 생략하고 메모리를 절약할 수 있습니다. 
- 매번 with 블록의 긴 코드를 적어주기는 귀찮으니 다음과 같이 no_grad라는 편의 함수를 준비했습니다. 

In [70]:
def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

- no_grad 함수는 단순히 using_config('enable_backprop', False)를 호출하는 코드를 return으로 돌려줄 뿐입니다. 
- 이제 기울기가 필요 없을 때는 no_grad 함수를 호출하면 됩니다. 
- 앞으로는 기울기 계산이 필요 없을 때, 즉 단순히 순전파 계산만 필요할 때는 방금 구현한 '모드 전환'을 사용하겠습니다. 

In [62]:
# 18장을 전체 코드로 정리한 버전

import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()
print(y.grad, t.grad)  # None None
print(x0.grad, x1.grad)  # 2.0 1.0


with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

None None
2.0 1.0


# 19. 변수 사용성 개선

- 이번 단계에서는 Variable 클래스를 더욱 쉽게 사용할 수 있게 해보겠습니다. 

## 19.1 변수 이름 지정

- 변수를 구분하기 위해 변수에 이름을 붙여주겠습니다. 
- Variable 클래스에 name이라는 인스턴스 변수를 추가했습니다. 
    - 변수에 이름을 붙일 수 있다면 계산 그래프를 시각화할 때 변수 이름을 그래프에 표시할 수 있습니다. 

In [ ]:
class Variable:
    def __init__(self, data, name=None):    # 변수를 구분하기 위해 name이라는 인스턴스 변수를 추가함, 아무 값도 주지 않으면 None
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name                    # 변수를 구분하기 위해 name이라는 인스턴스 변수를 추가함 
        self.grad = None
        self.creator = None
        self.generation = 0

## 19.2 ndarray 인스턴스 변수

- Variable은 데이터를 담는 '상자' 역할을 합니다. 
- 사용하는 사람 입장에서 중요한 것은 상자가 아니라 그 안의 '데이터'입니다. 
- 그래서 Variable이 데이터인 것처럼 보이게 하는 장치, 즉 상자를 투명하게 해주는 장치를 만들겠습니다. 
    - 머신러닝 시스템에서는 다차원 배열(텐서)를 기본 데이터 구조로 사용합니다.
    - 따라서 Variable 클래스는 (스칼라는 무시하고) ndarray만을 취급하기로 했습니다. 
    - 그래서 이번 절의 목표는 Variable 인스턴스를 ndarray 인스턴스 처럼 보이게 하는 것입니다. 
- Variable 안에는 ndarray 인스턴스가 있습니다. 
    - 넘파이의 ndarray 인스턴스에는 다차원 배열용 인스턴스 변수가 몇 가지 제공됩니다. 
    - 그 중 하나인 shape 인스턴스 변수를 사용하는 모습입니다. 

In [71]:
import numpy as np
x = np.array([[1, 2, 3], [4, 5, 6]])
x.shape   # 인스턴스 변수 shape은 다차원 배열의 형상을 알려줍니다. 

(2, 3)

In [73]:
class Variable:
...
    @property   # shape 메서드를 인스턴스 변수처럼 사용할 수 있게 됩니다. 
    def shape(self):  # shape 메서드를 추가한 후 실제 데이터의 shape을 반환하도록 했습니다. 
        return self.data.shape

In [77]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x.shape)  # x.shape() 대신 x.shape으로 호출할 수 있습니다. 

(2, 3)


- 이와 같이 메서드 호출이 아닌 인스턴스 변수로 데이터의 형상을 얻을 수 있습니다. 
- 같은 방법으로 ndarray의 다른 인스턴스 변수들을 Variable에 추가할 수 있습니다. 

In [79]:
class Variable:
...

    @property
    def ndim(self):  # 차원수
        return self.data.ndim

    @property
    def size(self):  # 원소수
        return self.data.size

    @property
    def dtype(self): # 데이터 타입, dtype을 지정하지 않으면 ndarray 인스턴스는 (환경에 따라) float64 또는 int64로 초기화 됨
        return self.data.dtype  # 신경망에서는 float32를 사용하는 경우가 많습니다. 

## 19.3 len 함수와 print 함수

- Variable 클래스를 더 확장하여 파이썬의 len 함수와도 함께 사용할 수 있도록 하겠습니다. 
- len은 객체 수를 알려주는 파이썬의 표준 함수 입니다. 
    - ndarray 인스턴스라면 첫 번째 차원의 원소 수를 반환합니다. 

In [80]:
x = [1, 2, 3, 4]
len(x)

4

In [81]:
x = np.array([1, 2, 3, 4])
len(x)

4

In [83]:
x = np.array([[1, 2, 3], [4, 5, 6]])
len(x)

2

In [84]:
class Variable:
...
    def __len__(self):   # __len__ 이라는 특수 메서드를 구현하면 Variable 인스턴스에 대해서도 len 함수를 사용할 수 있게 됩니다. 
        return len(self.data)

In [91]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(len(x))

2


- Variable 인스턴스를 print 함수에 건네면 안에 담긴 ndarray 인스턴스의 내용을 출력하도록 하겠습니다. 
- 이 때 출력 결과는 variable(...) 형태로 통일하여 사용자에게 Variable 인스턴스 임을 알려줍니다. 
- 값이 None 이거나 내용을 여러 줄로 출력해야 하는 경우도 지원합니다. 
- 여러 줄일 때는 공백 문자의 시작 위치를 조정하여 보기 좋게 출력합니다. 
- 다음은 이상의 조건을 만족하는 Variable의 $__repr__$ 메서드 모습입니다. 

In [ ]:
class Variable:
...
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'


- print 함수가 출력해 주는 문자열을 입맛에 맞게 정의하려면 $__repr__$ 메서드를 재정의 하면 됩니다. 
- 반환값은 출력하고자 하는 문자열입니다. 
- 앞의 코드에서는 str(self.data)를 이용하여 ndarray 인스턴스를 문자열로 변환했습니다. 
- str 함수 안에서는 ndarray 인스턴스의 $__str__$ 함수가 호출되고 숫자가 문자열로 변환됩니다. 
- 줄바꿈(\n)이 있으면 줄을 바꾼 후 새로운 줄 앞에 공백 9개를 삽입하여 여러 줄에 걸친 출력도 숫자의 시작 위치가 가지런하게 표시되게 했습니다. 
- 마지막으로 변환된 문자열을 'variable(...)' 형태로 감쌉니다. 
- 이상으로 Variable 클래스를 '투명한 상자'로 만드는 작업을 일부 끝마쳤습니다. 

In [93]:
x = Variable(np.array([1, 2, 3]))
print(x)
             
x = Variable(None)
print(x)
             
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x)

variable([1 2 3])
variable(None)
variable([[1 2 3]
          [4 5 6]])


In [90]:
# 19장을 전체 코드로 정리한 버전

import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
x.name = 'x'

print(x.name)
print(x.shape)
print(x)


x
(2, 3)
variable([[1 2 3]
          [4 5 6]])


# 20. 연산자 오버로드(1)

- Variable 인스턴스 a와 b가 있을 때 y = a * b처럼 코딩할 수 있으면 아주 유용한데, 이렇게 확장하는 것이 이번 단계의 목표입니다. 
    - 궁극적인 목표는 Variable 인스턴스를 ndarray 인스턴스 처럼 '보이게' 만드는 것입니다. 
    - 이렇게 하면 DeZero를 평범한 넘파이 코드를 작성하듯 사용할 수 있어서 넘파이를 사용해 본 사람들이 아주 쉽게 배울 수 있습니다. 
    
## 20.1 Mul 클래스 구현

- 이제부터 * 연산을 지원하도록 Variable을 확장할 것입니다. 
- 곱셈의 미분은 y = x0 * x1 일 때 아래와 같습니다. 

![title](20-1.png)

- 위에서 보듯 역전파는 최종 출력인 L의 미분을 정확하게는 L의 각 변수에 대한 미분을 전파합니다. 
- 우리는 스칼라를 출력하는 합성 함수에 관심이 있습니다. 
- 위 그림에서 마지막에 L이라는 스칼라를 출력하는 합성 함수를 가정했습니다. 
- 여기서 L은 오차, 다른 말로 손실 loss를 뜻합니다. 
- 다음과 같이 구현할 수 있습니다. 

In [ ]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0
    
def mul(x0, x1):  # Mul 클래스를 파이썬 함수로 사용할 수 있도록 해주는 코드입니다. 
    return Mul()(x0, x1)

In [96]:
# 이제 mul 함수를 사용하여 곱셈을 할 수 있습니다. 
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c)
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


## 20.2 연산자 오버로드

- y = a * b + c 형태로 사용하기 위해서 연산자 오버로드(operation overload)를 이용할 것입니다. 
    - 연산자를 오버로드하면 +와 *같은 연산자를 사용 시 사용자가 설정한 함수가 호출됩니다. 
    - 파이썬에서는 $__add__$와 $__mul__$ 같은 특수 메서드를 정의함으로써 사용자 지정 함수가 호출되도록 합니다. 

In [ ]:
Variable:
...

    def __mul__(self, other):
        return mul(self, other)

In [98]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
y = a * b
print(y)

variable(6.0)


- a * b가 실행될 때 인스턴스 a의 $__mul__(self.other)$ 메서드가 호출됩니다. 
- 아래 그림과 같이 연산자 * 왼쪽의 a가 인수 self에 전달되고, 오른쪽의 b가 other에 전달됩니다. 

![title](20-2.png)

- a * b가 실행되면 먼서 인스턴스 a의 특수 메서드인 $__mul__$이 호출됩니다. 
- 이 경우 b는 * 연산자의 오른쪽에 위치하기 때문에 $__mul__$ 이 아닌 $__rmul__$이라는 특수 메서드가 호둘됩니다. (r은 오른쪽을 의미)
- 이와 똑같은 작업을 아래 코드처럼 간단히 처리하는 방법도 있습니다. 

In [ ]:
class Variable:
    ...
    
Variable.__mul__ = mul
Variable.__add__ = add

- 파이썬에서는 함수도 객체이므로 이와 같이 함수 자체를 할당할 수 있습니다. 
- 이렇게 하면 Variable 인스턴스의 $__mul__$메서드를 호출할 때 mul 함수가 불립니다. 
- $+$ 연산자의 특수 메서드인 $__add__$도 설정해서 + 연산자도 함께 오버로드 해보겠습니다. 
- 그리고 $+$와 $*$를 모두 사용하여 계산을 해보겠습니다. 

In [100]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a, b), c)
y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [ ]:
- 이제 y = a * b + c 형태로 코딩하는 게 가능해졌습니다. 
- 계산 시 $+$와 $*$를 자유롭게 사용할 수 있게 되었습니다. 
- $/$와 $-$다른 

In [95]:
# 20장을 전체 코드로 정리한 버전

import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    return Mul()(x0, x1)


Variable.__add__ = add
Variable.__mul__ = mul

a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a, b), c)
y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)


variable(7.0)
2.0
3.0
